In [1]:
import os
import torch

from botorch.models.gp_regression import FixedNoiseGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.utils.transforms import unnormalize, normalize
from botorch.utils.sampling import draw_sobol_samples

from botorch.test_functions.multi_objective import BraninCurrin

from botorch.optim.optimize import optimize_acqf, optimize_acqf_list
from botorch.acquisition.objective import GenericMCObjective
from botorch.utils.multi_objective.scalarization import get_chebyshev_scalarization
from botorch.utils.multi_objective.box_decompositions.non_dominated import FastNondominatedPartitioning
from botorch.acquisition.multi_objective.monte_carlo import qExpectedHypervolumeImprovement, qNoisyExpectedHypervolumeImprovement
from botorch.utils.sampling import sample_simplex


import time
import warnings

from botorch import fit_gpytorch_mll
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils.multi_objective.box_decompositions.dominated import (
    DominatedPartitioning,
)
from botorch.utils.multi_objective.pareto import is_non_dominated


tkwargs = {
    "dtype": torch.double,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}
SMOKE_TEST = os.environ.get("SMOKE_TEST")

/Users/rileyhickman/mambaforge/envs/atlas/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

problem = BraninCurrin(negate=True).to(**tkwargs)
NOISE_SE = torch.tensor([15.19, 0.63], **tkwargs)

BATCH_SIZE = 4
NUM_RESTARTS = 10 if not SMOKE_TEST else 2
RAW_SAMPLES = 512 if not SMOKE_TEST else 4

standard_bounds = torch.zeros(2, problem.dim, **tkwargs)
standard_bounds[1] = 1

In [3]:
def generate_initial_data(n=6):
    # generate training data
    train_x = draw_sobol_samples(bounds=problem.bounds,n=n, q=1).squeeze(1)
    train_obj_true = problem(train_x)
    train_obj = train_obj_true + torch.randn_like(train_obj_true) * NOISE_SE
    return train_x, train_obj, train_obj_true


def initialize_model(train_x, train_obj):
    # define models for objective and constraint
    train_x = normalize(train_x, problem.bounds)
    models = []
    for i in range(train_obj.shape[-1]):
        train_y = train_obj[..., i:i+1]
        train_yvar = torch.full_like(train_y, NOISE_SE[i] ** 2)
        models.append(
            FixedNoiseGP(train_x, train_y, train_yvar, outcome_transform=Standardize(m=1))
        )
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    return mll, model

def optimize_qnehvi_and_get_observation(model, train_x, train_obj, sampler):
    """Optimizes the qEHVI acquisition function, and returns a new candidate and observation."""
    # partition non-dominated space into disjoint rectangles
    print(normalize(train_x, problem.bounds).shape)
    print(problem.bounds.shape)
    #print(standard_bounds)
    print(problem.ref_point.tolist())
    print(sampler)
    
    acq_func = qNoisyExpectedHypervolumeImprovement(
        model=model,
        ref_point=problem.ref_point.tolist(),  # use known reference point 
        X_baseline=normalize(train_x, problem.bounds),
        prune_baseline=True,  # prune baseline points that have estimated zero probability of being Pareto optimal
        sampler=sampler,
    )
    print(acq_func.objective)
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200},
        sequential=True,
    )
    # observe new values 
    new_x =  unnormalize(candidates.detach(), bounds=problem.bounds)
    new_obj_true = problem(new_x)
    new_obj = new_obj_true + torch.randn_like(new_obj_true) * NOISE_SE
    return new_x, new_obj, new_obj_true

In [4]:
warnings.filterwarnings("ignore", category=BadInitialCandidatesWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

N_BATCH = 20 if not SMOKE_TEST else 10
MC_SAMPLES = 128 if not SMOKE_TEST else 16

verbose = True

hvs_qparego, hvs_qehvi, hvs_qnehvi, hvs_random = [], [], [], []

In [5]:
train_x_qparego, train_obj_qparego, train_obj_true_qparego = generate_initial_data(
    n=2 * (problem.dim + 1)
)
train_x_qnehvi, train_obj_qnehvi, train_obj_true_qnehvi = (
    train_x_qparego,
    train_obj_qparego,
    train_obj_true_qparego,
)
mll_qnehvi, model_qnehvi = initialize_model(train_x_qnehvi, train_obj_qnehvi)

# compute hypervolume
bd = DominatedPartitioning(ref_point=problem.ref_point, Y=train_obj_true_qparego)
volume = bd.compute_hypervolume().item()

hvs_qnehvi.append(volume)

In [6]:
# run N_BATCH rounds of BayesOpt after the initial random batch
for iteration in range(1, N_BATCH + 1):

    t0 = time.monotonic()

    # fit the models
    fit_gpytorch_mll(mll_qnehvi)

    # define the qEI and qNEI acquisition modules using a QMC sampler
    qnehvi_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([MC_SAMPLES]))

    # optimize acquisition functions and get new observations
    (
        new_x_qnehvi,
        new_obj_qnehvi,
        new_obj_true_qnehvi,
    ) = optimize_qnehvi_and_get_observation(
        model_qnehvi, train_x_qnehvi, train_obj_qnehvi, qnehvi_sampler
    )


    # update training points
 
    train_x_qnehvi = torch.cat([train_x_qnehvi, new_x_qnehvi])
    train_obj_qnehvi = torch.cat([train_obj_qnehvi, new_obj_qnehvi])
    train_obj_true_qnehvi = torch.cat([train_obj_true_qnehvi, new_obj_true_qnehvi])

    print(train_obj_true_qnehvi)
    print(train_obj_true_qnehvi.shape)
    
    print('ref point : ', problem.ref_point)
    # compute hypervolume
    bd = DominatedPartitioning(ref_point=problem.ref_point, Y=train_obj_true_qnehvi)
    volume = bd.compute_hypervolume().item()
    hvs_qnehvi.append(volume)

    # reinitialize the models so they are ready for fitting on next iteration
    # Note: we find improved performance from not warm starting the model hyperparameters
    # using the hyperparameters from the previous iteration
    mll_qnehvi, model_qnehvi = initialize_model(train_x_qnehvi, train_obj_qnehvi)

    t1 = time.monotonic()
    
    print(f'Batch {iteration}: Hypervolume : {hvs_qnehvi[-1]}')

torch.Size([6, 2])
torch.Size([2, 2])
[-18.0, -6.0]
SobolQMCNormalSampler()
IdentityMCMultiOutputObjective()
tensor([[-199.9686,   -4.2916],
        [ -68.7968,  -13.6896],
        [ -49.0098,   -6.3551],
        [  -8.6178,   -8.2191],
        [ -74.6644,   -6.1903],
        [ -21.3658,   -8.4658],
        [ -17.5083,   -1.1804],
        [ -49.3868,   -5.4222],
        [ -10.9609,  -10.1795],
        [ -35.6473,   -5.8349]], dtype=torch.float64)
torch.Size([10, 2])
ref point :  tensor([-18.,  -6.], dtype=torch.float64)
Batch 1: Hypervolume : 2.369795709893773
torch.Size([10, 2])
torch.Size([2, 2])
[-18.0, -6.0]
SobolQMCNormalSampler()
IdentityMCMultiOutputObjective()
tensor([[-199.9686,   -4.2916],
        [ -68.7968,  -13.6896],
        [ -49.0098,   -6.3551],
        [  -8.6178,   -8.2191],
        [ -74.6644,   -6.1903],
        [ -21.3658,   -8.4658],
        [ -17.5083,   -1.1804],
        [ -49.3868,   -5.4222],
        [ -10.9609,  -10.1795],
        [ -35.6473,   -5.8349],
   

tensor([[-199.9686,   -4.2916],
        [ -68.7968,  -13.6896],
        [ -49.0098,   -6.3551],
        [  -8.6178,   -8.2191],
        [ -74.6644,   -6.1903],
        [ -21.3658,   -8.4658],
        [ -17.5083,   -1.1804],
        [ -49.3868,   -5.4222],
        [ -10.9609,  -10.1795],
        [ -35.6473,   -5.8349],
        [ -39.3380,   -1.3918],
        [ -61.5713,   -1.5621],
        [ -26.0654,   -1.2763],
        [  -4.9312,   -5.6619],
        [  -5.9462,   -3.1995],
        [  -8.8739,  -11.0365],
        [  -3.2414,   -4.2447],
        [-145.8722,   -4.0053],
        [-150.1137,   -2.2393],
        [  -5.7458,   -4.7563],
        [  -4.2333,   -3.5638],
        [ -12.3924,   -2.5472],
        [ -19.1255,  -10.4773],
        [ -10.2468,   -2.2177],
        [ -20.8332,   -1.2218],
        [ -47.8799,   -6.2558],
        [ -13.5877,   -1.7040],
        [  -7.0846,   -2.8013],
        [  -3.6275,   -4.1519],
        [ -15.4317,   -1.4489],
        [ -21.8658,   -7.8301],
        

tensor([[-199.9686,   -4.2916],
        [ -68.7968,  -13.6896],
        [ -49.0098,   -6.3551],
        [  -8.6178,   -8.2191],
        [ -74.6644,   -6.1903],
        [ -21.3658,   -8.4658],
        [ -17.5083,   -1.1804],
        [ -49.3868,   -5.4222],
        [ -10.9609,  -10.1795],
        [ -35.6473,   -5.8349],
        [ -39.3380,   -1.3918],
        [ -61.5713,   -1.5621],
        [ -26.0654,   -1.2763],
        [  -4.9312,   -5.6619],
        [  -5.9462,   -3.1995],
        [  -8.8739,  -11.0365],
        [  -3.2414,   -4.2447],
        [-145.8722,   -4.0053],
        [-150.1137,   -2.2393],
        [  -5.7458,   -4.7563],
        [  -4.2333,   -3.5638],
        [ -12.3924,   -2.5472],
        [ -19.1255,  -10.4773],
        [ -10.2468,   -2.2177],
        [ -20.8332,   -1.2218],
        [ -47.8799,   -6.2558],
        [ -13.5877,   -1.7040],
        [  -7.0846,   -2.8013],
        [  -3.6275,   -4.1519],
        [ -15.4317,   -1.4489],
        [ -21.8658,   -7.8301],
        

tensor([[-199.9686,   -4.2916],
        [ -68.7968,  -13.6896],
        [ -49.0098,   -6.3551],
        [  -8.6178,   -8.2191],
        [ -74.6644,   -6.1903],
        [ -21.3658,   -8.4658],
        [ -17.5083,   -1.1804],
        [ -49.3868,   -5.4222],
        [ -10.9609,  -10.1795],
        [ -35.6473,   -5.8349],
        [ -39.3380,   -1.3918],
        [ -61.5713,   -1.5621],
        [ -26.0654,   -1.2763],
        [  -4.9312,   -5.6619],
        [  -5.9462,   -3.1995],
        [  -8.8739,  -11.0365],
        [  -3.2414,   -4.2447],
        [-145.8722,   -4.0053],
        [-150.1137,   -2.2393],
        [  -5.7458,   -4.7563],
        [  -4.2333,   -3.5638],
        [ -12.3924,   -2.5472],
        [ -19.1255,  -10.4773],
        [ -10.2468,   -2.2177],
        [ -20.8332,   -1.2218],
        [ -47.8799,   -6.2558],
        [ -13.5877,   -1.7040],
        [  -7.0846,   -2.8013],
        [  -3.6275,   -4.1519],
        [ -15.4317,   -1.4489],
        [ -21.8658,   -7.8301],
        